<a href="https://colab.research.google.com/github/abhi25031998/Abhishek_INFO5731_Spring2025/blob/main/Guruvinamath_Abhishek_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
!pip install requests beautifulsoup4 pandas
!pip install selenium webdriver-manager pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 44.2 MB/s eta 0:00:00


In [67]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import pandas as pd
import time

base_url = "https://ddr.densho.org/narrators/"

# get narrators
def get_narrators(url):
    headers = {
        "User-Agent": "AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36"
    }
    try:
        req = Request(url, headers=headers)
        response = urlopen(req)
        soup = BeautifulSoup(response, "html.parser")

        narrators = []
        narrator_containers = soup.find_all(class_='media-body')
        for narrator in narrator_containers:
            name = narrator.find("a").get_text(strip=True)
            details = narrator.find(class_="source muted").get_text(strip=True)
            narrators.append({"Name": name, "Details": details})

        return narrators
    except HTTPError as e:
        print(f"HTTP Error: {e}")
        return []

# collect narrators
def collect_narrators(limit=904):
    all_narrators = []
    page = 1
    while True:
        url = f"{base_url}?page={page}"
        print(f"Fetching narrators from page {page}...")
        narrators = get_narrators(url)
        if not narrators:
            print("No more narrators found.")
            break
        all_narrators.extend(narrators)
        page += 1
        if len(all_narrators) >= limit:
            break

        time.sleep(2)

    return all_narrators[:limit]
narrators = collect_narrators()
df = pd.DataFrame(narrators)
df.to_csv('densho_repository_narrators.csv', index=False)

print(f"Collected {len(narrators)} narrators and saved to 'Densho_Repository_narrators.csv'.")

Fetching narrators from page 1...
Fetching narrators from page 2...
Fetching narrators from page 3...
Fetching narrators from page 4...
Fetching narrators from page 5...
Fetching narrators from page 6...
Fetching narrators from page 7...
Fetching narrators from page 8...
Fetching narrators from page 9...
Fetching narrators from page 10...
Fetching narrators from page 11...
Fetching narrators from page 12...
Fetching narrators from page 13...
Fetching narrators from page 14...
Fetching narrators from page 15...
Fetching narrators from page 16...
Fetching narrators from page 17...
Fetching narrators from page 18...
Fetching narrators from page 19...
Fetching narrators from page 20...
Fetching narrators from page 21...
Fetching narrators from page 22...
Fetching narrators from page 23...
Fetching narrators from page 24...
Fetching narrators from page 25...
Fetching narrators from page 26...
Fetching narrators from page 27...
Fetching narrators from page 28...
Fetching narrators from page 

# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [68]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

df = pd.read_csv('densho_repository_narrators.csv')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def cleaned_text(text):
    # Handle NaN values
    if pd.isna(text):
        return "", ""
    # (1) Remove noise (special characters and punctuations)
    text = re.sub(r'[^\w\s]', '', text)

    # (2) Remove numbers
    text = re.sub(r'\d+', '', text)

    # (3) Remove stopwords
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])

    # (4) Lowercase all texts
    text = text.lower()

    # (5) Stemming
    stemmed_text = ' '.join([stemmer.stem(word) for word in text.split()])

    # (6) Lemmatization
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return stemmed_text, lemmatized_text

df[['Stemmed_Details', 'Lemmatized_Details']] = df['Details'].apply(clean_text).apply(pd.Series)

df.to_csv('cleaned_densho_repository_narrators.csv', index=False)

print("Text cleaning completed. Cleaned data saved to 'cleaned_densho_repository_narrators.csv'.")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Text cleaning completed. Cleaned data saved to 'cleaned_densho_repository_narrators.csv'.


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [54]:
import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.tree import Tree
from nltk.chunk import ne_chunk
import spacy
from collections import Counter


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')


nlp = spacy.load("en_core_web_sm")


df = pd.read_csv('cleaned_densho_repository_narrators.csv')

def pos_tagging_and_count(text):
    if pd.isna(text):
        return [], Counter()
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    pos_counts = Counter(tag for word, tag in pos_tags)
    return pos_tags, pos_counts


def constituency_parsing(text):
    if pd.isna(text):
        return ""
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    tree = ne_chunk(pos_tags)
    return tree


def dependency_parsing(text):
    if pd.isna(text):
        return []
    doc = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in doc]


def named_entity_recognition(text):
    if pd.isna(text):
        return []
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

for index, row in df.iterrows():
    text = row['Lemmatized_Details']
    if pd.isna(text):
        print(f"\nSkipping Row {index + 1}: Empty text")
        print("\n" + "=" * 80 + "\n")
        continue

    print(f"\nAnalyzing Row {index + 1}: {text}")

    # (1) POS Tagging
    pos_tags, pos_counts = pos_tagging_and_count(text)
    print("\nPOS Tags:")
    print(pos_tags)
    print("\nPOS Counts:")
    print(f"Nouns (N): {pos_counts.get('NN', 0) + pos_counts.get('NNS', 0) + pos_counts.get('NNP', 0) + pos_counts.get('NNPS', 0)}")
    print(f"Verbs (V): {pos_counts.get('VB', 0) + pos_counts.get('VBD', 0) + pos_counts.get('VBG', 0) + pos_counts.get('VBN', 0) + pos_counts.get('VBP', 0) + pos_counts.get('VBZ', 0)}")
    print(f"Adjectives (Adj): {pos_counts.get('JJ', 0) + pos_counts.get('JJR', 0) + pos_counts.get('JJS', 0)}")
    print(f"Adverbs (Adv): {pos_counts.get('RB', 0) + pos_counts.get('RBR', 0) + pos_counts.get('RBS', 0)}")

    # (2) Constituency Parsing
    print("\nConstituency Parsing Tree:")
    tree = constituency_parsing(text)
    if isinstance(tree, Tree):
        tree.pretty_print()
    else:
        print(tree)

    # (3) Dependency Parsing
    print("\nDependency Parsing:")
    dependencies = dependency_parsing(text)
    for dep in dependencies:
        print(f"Token: {dep[0]}, Dependency: {dep[1]}, Head: {dep[2]}")

    # (4) Named Entity Recognition
    print("\nNamed Entity Recognition:")
    entities = named_entity_recognition(text)
    entity_counts = Counter(label for text, label in entities)
    print("Entities Found:")
    for entity, label in entities:
        print(f"Entity: {entity}, Label: {label}")
    print("\nEntity Counts:")
    for label, count in entity_counts.items():
        print(f"{label}: {count}")

    print("\n" + "=" * 80 + "\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_

Streaming output truncated to the last 5000 lines.
Nouns (N): 10
Verbs (V): 3
Adjectives (Adj): 2
Adverbs (Adv): 0

Constituency Parsing Tree:
                                                                        S                                                                                       
    ____________________________________________________________________|________________________________________________________________________________        
nisei/JJ female/NN born/NN may/MD seattle/VB washington/NN grew/VBD seattle/JJ world/NN war/NN ii/NN removed/VBD puyallup/NN assembly/NN center/NN washington/NN


Dependency Parsing:
Token: nisei, Dependency: amod, Head: female
Token: female, Dependency: nsubj, Head: grew
Token: born, Dependency: acl, Head: female
Token: may, Dependency: aux, Head: grew
Token: seattle, Dependency: compound, Head: washington
Token: washington, Dependency: nsubj, Head: grew
Token: grew, Dependency: ROOT, Head: grew
Token: seattle, Dependency: comp

# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [64]:
#Part 1
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re
import pandas as pd
import csv

NoOfPages = 55
Dlist = []
with open('github_marketplace_products.csv', mode='w', newline='', encoding='utf-8') as file:
  mywriter = csv.DictWriter(file, fieldnames=['Product Name', 'Description', 'URL', 'Page Number'])
  mywriter.writeheader()
  for j in range(1,NoOfPages+1):
    Git_marketplace_URL = f"https://github.com/marketplace?page={j}&type=actions"

    Git_Link = Request(Git_marketplace_URL, headers={'User-Agent': 'AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36'})
    Git_URL = urlopen(Git_Link)

    Git_Response = Git_URL.read()

    Git_Soup = BeautifulSoup(Git_Response,"html.parser")


    Action_Div = Git_Soup.find_all(class_="position-relative border rounded-2 d-flex marketplace-common-module__marketplace-item--MohVH gap-3 p-3")

    for i in Action_Div:
      name_tag = i.find(class_="marketplace-common-module__marketplace-item-link--jrIHf line-clamp-1")
      productName = name_tag.get_text().strip()
      desc_tag = i.find('p')
      productDescription = desc_tag.get_text().strip()
      URL_tag = name_tag['href']
      ProductURL = 'https://github.com/'+str(URL_tag)
      mywriter.writerow({
                'Product Name': productName,
                'Description': productDescription,
                'URL': ProductURL,
                'Page Number': j
            })

    time.sleep(2)


In [65]:
#Part 2
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


git_marketplace_df = pd.read_csv('github_marketplace_products.csv')

#data preprocessing
# Initialize Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def cleaning_text(text):
    # Convert non-string types to string
    if not isinstance(text, str):
        text = str(text)

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and digits (keeping only letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and apply lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    return ' '.join(tokens)

git_marketplace_df['Description'] = git_marketplace_df['Description'].apply(preprocess_text)


#data quality checking

# 1. Check for missing values
missing_data = git_marketplace_df.isnull().sum()

# 2. Remove duplicates based on 'Product Name' and 'URL' columns
git_marketplace_df = git_marketplace_df.drop_duplicates(subset=['Product Name', 'URL'])

# 3. Ensure that 'Product Name' and 'URL' are not empty
git_marketplace_df = git_marketplace_df[git_marketplace_df['Product Name'].notna() & git_marketplace_df['URL'].notna()]

# 4. Check for invalid URLs (basic check)
url_pattern = re.compile(r'^(https?://)?(www\.)?github\.com/.+')
git_marketplace_df['Valid_URL'] = git_marketplace_df['URL'].apply(lambda x: bool(url_pattern.match(x)))

# Filter out rows with invalid URLs
git_marketplace_df = git_marketplace_df[git_marketplace_df['Valid_URL']]

# Save the cleaned data to a new CSV file
git_marketplace_df.to_csv('cleaned_github_marketplace_products.csv', index=False)


if missing_data.any():
    print("There is missing data.")
else:
    print("No missing data.")

print("\nCleaned Data Sample:")
print(git_marketplace_df.head())

len(git_marketplace_df)

No missing data.

Cleaned Data Sample:
                   Product Name  \
0                TruffleHog OSS   
1                 Metrics embed   
2  yq - portable yaml processor   
3                  Super-Linter   
4        Gosec Security Checker   

                                         Description  \
0                      scan github action trufflehog   
1  infographics generator plugins option display ...   
2      create read update delete merge validate yaml   
3  superlinter readytorun collection linters code...   
4                         run gosec security checker   

                                                 URL  Page Number  Valid_URL  
0  https://github.com//marketplace/actions/truffl...            1       True  
1  https://github.com//marketplace/actions/metric...            1       True  
2  https://github.com//marketplace/actions/yq-por...            1       True  
3  https://github.com//marketplace/actions/super-...            1       True  
4  https://github.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1026

#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [46]:
import tweepy

# Set your keys and tokens here
my_api_key = 'rKMU3qNXbQBYtDgEKJcRZYe5P'
my_api_secret_key = 's0iWt4nTMb6HVqA6U1yBk7jBnYuY7s79l065YZaT6LaXhmYzpv'
my_access_token = '1892383599816822784-C10T2HXrl5O1CW3rNv652ID6rapmnF'
my_access_secret_token = 'opu7rlUsMnIKWRsyt2Trasy0mPqQD2kWa8yF7IzzEDaHS'

# Authenticate with Twitter
auth = tweepy.OAuth1UserHandler(
    consumer_key=my_api_key,
    consumer_secret=my_api_secret_key,
    access_token=my_access_token,
    access_token_secret=my_access_secret_token
)
api = tweepy.API(auth)
tweets_limit = 100
hashtag = '#LLMs -is:retweet'

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAALmlzQEAAAAAao8KfYqppqD2aOEfeFBw1lPaGKM%3DWhMLku862jJhQUlHOJjls2QKnAJAFCEI2jUyLVXvlR3bnYculA')
tweets = client.search_recent_tweets(query=hashtag, tweet_fields=["created_at", "text", "author_id"], max_results=100)

In [47]:
# Create a dictionary to store tweet data
mytweetDict= {'tweet_id': [],'user_id': [],'tweet_time': [],'tweetText': []}

if tweets.data:
    for i in tweets.data:
        mytweetDict['tweet_id'].append(i.id)
        mytweetDict['user_id'].append(i.author_id)
        mytweetDict['tweet_time'].append(i.created_at)
        mytweetDict['tweetText'].append(i.text)

# dataframe creation
myTweets_df = pd.DataFrame(mytweetDict)

# Display the DataFrame
print(myTweets_df.head())

# Save DataFrame to a CSV file
myTweets_df.to_csv("LLMs_Tweets.csv", index=False)

              tweet_id              user_id                tweet_time  \
0  1892387115855446273  1241749653013630976 2025-02-20 01:33:41+00:00   
1  1892383265270489567  1667995528490020867 2025-02-20 01:18:23+00:00   
2  1892378135657926828   737142202481016832 2025-02-20 00:58:00+00:00   
3  1892375804622901331  1671957938884472832 2025-02-20 00:48:45+00:00   
4  1892374775265448353  1518654527511347200 2025-02-20 00:44:39+00:00   

                                           tweetText  
0  ▶️How #LLMs Work: Pre-Training to Post-Trainin...  
1  🧵8/n\nFor more details, check out our paper an...  
2  @enilev @iamVictorDey @observer @jblefevre60 @...  
3  DeepSeek’s NSA Outperforms Full Attention, Mak...  
4  Curator, an open source library that has 10x’d...  


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

This assignment was really helpful for me in learning webscraping. I really enjoyed learning how to webscrape and collect data from websites. The challenges I faced was during webscraping for amazon , imdb etc in question 1, http parsing request was getting blocked. So, kind of took a long time to code. And codes were little challenging and advanced for me but I found the time given for the assignment completion would be enough to do it.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog